<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/01-Mi-segundo-modelo-Plantilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Facultad de Ciencias Exactas, Naturales y Ambientales</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Mi segundo modelo
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Alisson Manosalvas &bull; 29 Nov 2025
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

Este cuaderno está diseñado practicar en el entrenamiento y evaluación de un modelo de regresión lineal utilizando datos reales. Los objetivos principales son:

- Entrenar un modelo de regresión lineal.
- Evaluar el ajuste del modelo.


Del conjunto de datos disponible [aquí](https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv), se desea predecir la variable `price` en función de las variables `bedrooms`, `bathrooms`, `sqft_living`, `sqft_lot`, `floors`.

In [10]:
# Paquetes necesarios
import numpy as np
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, RobustScaler, StandardScaler, Normalizer, QuantileTransformer, PowerTransformer
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import make_scorer, mean_squared_error


---
## <font color='264CC7'> Lectura de Datos </font>


Realiza la lectura de los datos y presenta las primeras filas del conjunto de datos.

In [3]:
url = "https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv"
df = pd.read_csv(url)
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


---
## <font color='264CC7'> Preprocesamiento de datos </font>

Realiza un escalado de los datos, prueba diferentes técnicas y selecciona la que mejor se ajuste a los datos.

In [4]:
#Variables de interés
X = df[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors']]
Y = df['price']

In [5]:
# Normalización por el máximo
x_max = X.max()
X_max_normalized = X / x_max
print("Normalización por el máximo:\n")
display(X_max_normalized.head())

# Normalización Min-Max
min_max_scaler = MinMaxScaler()
X_min_max_normalized = pd.DataFrame(min_max_scaler.fit_transform(X), columns=X.columns)
print("Normalización Min-Max:\n")
display(X_min_max_normalized.head())

# Estandarización
standard_scaler = StandardScaler()
X_standardized = pd.DataFrame(standard_scaler.fit_transform(X), columns=X.columns)
print("Estandarización:\n")
display(X_standardized.head())

Normalización por el máximo:



,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.090909,0.12500,0.087149,0.003421,0.285714
1,0.090909,0.28125,0.189808,0.004385,0.571429
2,0.060606,0.12500,0.056869,0.006056,0.285714
3,0.121212,0.37500,0.144756,0.003028,0.285714
4,0.090909,0.25000,0.124077,0.004893,0.285714


Normalización Min-Max:



,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.090909,0.12500,0.067170,0.003108,0.0
1,0.090909,0.28125,0.172075,0.004072,0.4
2,0.060606,0.12500,0.036226,0.005743,0.0
3,0.121212,0.37500,0.126038,0.002714,0.0
4,0.090909,0.25000,0.104906,0.004579,0.0


Estandarización:



,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,-0.398737,-1.447464,-0.979835,-0.228321,-0.915427
1,-0.398737,0.175607,0.533634,-0.189885,0.936506
2,-1.473959,-1.447464,-1.426254,-0.123298,-0.915427
3,0.676485,1.149449,-0.130550,-0.244014,-0.915427
4,-0.398737,-0.149007,-0.435422,-0.169653,-0.915427


---
## <font color='264CC7'> Características y etiquetas </font>

Separa las características y las etiquetas y mostrar la dimensión de cada elemento.

In [7]:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors']
X = df[features]
y = df['price']

---
## <font color='264CC7'> Conjunto de entrenamiento y prueba </font>

Divide los datos en dos conjuntos: uno para entrenamiento y otro para prueba y visualiza la dimensión de cada conjunto. Usar `random_state` igual al últmo número de tu cédula.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

Revisa la distribución de las etiquetas en los conjuntos de entrenamiento y prueba.

In [12]:
print('Dimensiones de los conjuntos de datos:')
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

Dimensiones de los conjuntos de datos:
X_train: (17290, 5)
y_train: (17290,)
X_test: (4323, 5)
y_test: (4323,)


---
## <font color='264CC7'> Entrenamiento </font>

Define un modelo de regresión lineal y entrenalo con los datos de entrenamiento.

In [13]:
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [15]:
print("Ecuación de la recta:")
print(modelo.intercept_)
print(modelo.coef_)

Ecuación de la recta:
80382.98549304518
[-5.60573250e+04  5.92604497e+03  3.09525201e+02 -3.62496335e-01
 -1.26782813e+03]


---
## <font color='264CC7'> Evaluación </font>

Realiza la predicciones con el conjunto de prueba y muestra los valores en una tabla.

In [16]:
y_pred = modelo.predict(X_test)

In [17]:
df_predicciones = pd.DataFrame({
    'Actual': y_test.values.flatten(),
    'Predicción': y_pred.flatten()})
display(df_predicciones)

,Actual,Predicción
0,262000.0,3.275199e+05
1,593777.0,3.849087e+05
2,405000.0,4.377430e+05
3,366000.0,4.866932e+05
4,361000.0,3.727038e+05
...,...,...
4318,645000.0,5.105418e+05
4319,299950.0,4.737597e+05
4320,1484900.0,1.219927e+06
4321,263000.0,4.136184e+05


Calcula el error cuadrático medio.

In [19]:
mse = mean_squared_error(y_test, y_pred)

In [20]:
print('Error cuadrático medio:', mse)

Error cuadrático medio: 65009322812.335846


Escribe el error cuadrático medio para cada tipo de escalado:

- Escalado MaxAbsScaler:
- Escalado MinMaxScaler:
- Escalado RobustScaler:
- Escalado StandardScaler:
- Escalado Normalizer:
- Escalado QuantileTransformer:
- Escalado PowerTransformer:

Escribe un párrafo sobre qué hace cada escalado no visto en clase:

- **Escalado ...:** ...

In [21]:
normalizaciones = {
    "MaxAbsScaler": MaxAbsScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "RobustScaler": RobustScaler(),
    "StandardScaler": StandardScaler(),
    "Normalizer": Normalizer(),
    "QuantileTransformer": QuantileTransformer(output_distribution='normal'),
    "PowerTransformer": PowerTransformer()
}

resultados = {}

for nombre, scaler in normalizaciones.items():
    scaler.fit(X_train)
    X_train_norm = scaler.transform(X_train)
    X_test_norm = scaler.transform(X_test)

    modelo = LinearRegression()
    modelo.fit(X_train_norm, y_train)

    y_pred = modelo.predict(X_test_norm)
    mse = mean_squared_error(y_test, y_pred)

    resultados[nombre] = mse

for nombre, mse in resultados.items():
    print(f"Normalización {nombre}: {mse}")

Normalización MaxAbsScaler: 65009322812.33681
Normalización MinMaxScaler: 65009322812.3368
Normalización RobustScaler: 65009322812.33679
Normalización StandardScaler: 65009322812.3368
Normalización Normalizer: 107785576857.27591
Normalización QuantileTransformer: 82298705653.15262
Normalización PowerTransformer: 82231486222.29245


**QuantileTransformer**

El QuantileTransformer transforma los valores de cada característica siguiendo la posición que ocupan dentro de su distribución, convirtiéndolos en una forma completamente uniforme o aproximadamente normal. Al redistribuir los valores según sus cuantiles, logra como aplanar concentraciones de datos y ubicar los valores atípicos dentro de una escala más equilibrada.

Esto es útil cuando los datos presentan fuertes sesgos o colas largas, ya que permite que modelos sensibles a la forma de la distribución trabajen sobre un espacio más estable y homogéneo

**PowerTransformer**

Aplica transformaciones matemáticas basadas en potencias como Box-Cox o Yeo Johnson para que las variables adopten una forma más parecida a la distribución normal. Con esta técnica se reducen asimetrías, se estabiliza la varianza y se suavizan diferencias extremas entre valores pequeños y grandes. Este tipo de normalización beneficia a los modelos que asumen datos simétricos o con dispersión constante, ayudando a mejorar la linealidad de las relaciones y el desempeño general del modelo.

**RobustScaler**

Usa medidas estadísticas resistentes a valores extremos, en lugar de centrar los datos en torno a la media, se usa la mediana, y en lugar de la desviación estándar, emplea el rango intercuartílico (IQR). Con esto, la escala de las características queda determinada por la región donde se concentra la mayoría de los datos, ignorando la influencia de los outliers.

Es una técnica ideal cuando el conjunto de datos tiene valores muy alejados del resto, ya que evita que estos dominen el proceso de escalado y preserva la estructura esencial de las variables.

**Normalizer**

El Normalizer transforma cada muestra cada fila para que su vector de características tenga una norma igual a uno. Esto significa que no trabaja a nivel de columnas sino de registros individuales, reescalando cada observación según su propia magnitud. Lo importante no es el tamaño de los valores, sino la proporción entre ellos dentro de cada muestra. Por eso se utiliza en contextos donde la dirección del vector es más relevante que su longitud, como en análisis de similitud de documentos o en algoritmos de clustering basados en ángulos.